In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train=pd.read_csv('/kaggle/input/titanic/train.csv')

In [3]:
train=train.drop(columns=['PassengerId','Name','Ticket','Cabin'])

In [4]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset


In [5]:
sex_unique=train['Sex'].unique()
sex_id={label:id for id,label in enumerate(sex_unique)}
train["Sex"]=train["Sex"].map(sex_id)
train["Age"] = train["Age"].fillna(train["Age"].mode()[0])
train["Fare"]=train["Fare"].fillna(train["Fare"].mode()[0])
train=train.dropna()
embarked_uniq=train["Embarked"].unique()
emb_id={label:id for id,label in enumerate(embarked_uniq)}
train["Embarked"]=train["Embarked"].map(emb_id)
train

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,22.0,1,0,7.2500,0
1,1,1,1,38.0,1,0,71.2833,1
2,1,3,1,26.0,0,0,7.9250,0
3,1,1,1,35.0,1,0,53.1000,0
4,0,3,0,35.0,0,0,8.0500,0
...,...,...,...,...,...,...,...,...
886,0,2,0,27.0,0,0,13.0000,0
887,1,1,1,19.0,0,0,30.0000,0
888,0,3,1,24.0,1,2,23.4500,0
889,1,1,0,26.0,0,0,30.0000,1


In [6]:
from torch.utils.data import Dataset
import torch
from sklearn.preprocessing import StandardScaler

class Tit(Dataset):
    def __init__(self, df, scaler=None, fit_scaler=False):
        super().__init__()

        # ✅ Target
        self.y = torch.tensor(
            df["Survived"].values,
            dtype=torch.float32
        ).view(-1, 1)

        # ✅ Features from df (same as you wanted)
        X = df.drop(columns="Survived").values

        # ✅ Scale correctly
        if fit_scaler:
            self.scaler = StandardScaler()
            X = self.scaler.fit_transform(X)   # ✅ FIT ONLY HERE (TRAIN)
        else:
            self.scaler = scaler               # ✅ REUSE for test/inference
            X = self.scaler.transform(X)

        # ✅ Convert to Torch AFTER scaling
        self.x = torch.tensor(X, dtype=torch.float32)

    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return len(self.x)


In [7]:
data=Tit(train,fit_scaler=True)
scaler=data.scaler
print(data[0][0])

tensor([ 0.8252, -0.7353, -0.4955,  0.4314, -0.4743, -0.5002, -0.5697])


In [8]:
dataloader=DataLoader(data,batch_size=64,shuffle=True)


In [9]:
import torch
import torch.nn as nn

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.net = nn.Sequential(
            nn.Linear(7, 64),
            nn.LeakyReLU(),
            nn.Dropout(0.5),

            nn.Linear(64, 128),
            nn.LeakyReLU(),
            nn.Dropout(0.5),

            nn.Linear(128, 32),
            nn.LeakyReLU(),

            nn.Linear(32, 1) 
        )

    def forward(self, x):
        return self.net(x)


In [47]:
model.net[2].p=0
model.net[5].p=0

In [48]:

print(model)

Model(
  (net): Sequential(
    (0): Linear(in_features=7, out_features=64, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Dropout(p=0, inplace=False)
    (3): Linear(in_features=64, out_features=128, bias=True)
    (4): LeakyReLU(negative_slope=0.01)
    (5): Dropout(p=0, inplace=False)
    (6): Linear(in_features=128, out_features=32, bias=True)
    (7): LeakyReLU(negative_slope=0.01)
    (8): Linear(in_features=32, out_features=1, bias=True)
  )
)


In [11]:
optimizer=torch.optim.AdamW(model.parameters(),lr=1e-4,weight_decay=0.1)
lossy=nn.BCEWithLogitsLoss()

In [12]:
import joblib

In [13]:
device=torch.device("cpu")

In [53]:
model.train()
bloss=float('inf')
for i in range(10000000):
    tloss=0
    for _,(inp,out) in enumerate(dataloader):
        inp=inp.to(device)
        out=out.to(device)
        y_pred=model(inp)
        loss=lossy(y_pred,out)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        tloss+=loss.item()
    print(f'Epoch {i+1} has loss of {tloss/len(dataloader)}') if (i+1)%100==0 else None
    if tloss<bloss:
        bloss=tloss
        print(f"Saved at {bloss/len(dataloader)}")
        joblib.dump(model.state_dict(),'model.pkl')

Saved at 0.14930984643953188
Saved at 0.14839924073645047
Saved at 0.14828849477427347
Saved at 0.14805571947778975
Saved at 0.14800454995461873
Saved at 0.14673738128372601
Epoch 100 has loss of 0.14802540625844682
Saved at 0.1466862644468035
Saved at 0.14586532648120606
Saved at 0.14577342942357063
Saved at 0.14536339097789355
Epoch 200 has loss of 0.14877210876771382
Saved at 0.1451228495155062
Saved at 0.14449928275176457
Epoch 300 has loss of 0.14680591065968787
Saved at 0.14427935119186128
Saved at 0.14421146815376623
Saved at 0.14362708372729166
Saved at 0.14357330277562141
Epoch 400 has loss of 0.14517629838415555
Saved at 0.14314664314900125
Saved at 0.1430437889482294
Epoch 500 has loss of 0.14321802822606905
Saved at 0.14259229227900505
Saved at 0.14253341725894383
Saved at 0.14238196717841284
Saved at 0.14197338665170328
Epoch 600 has loss of 0.14327446797064372
Saved at 0.14179687627724238
Saved at 0.14149299530046328
Saved at 0.14065861063344137
Epoch 700 has loss of 0.14

KeyboardInterrupt: 

In [52]:
for param_group in optimizer.param_groups:
    param_group["lr"] = 1e-4

In [ ]:
print("Current LR:", optimizer.param_groups[0]["lr"])


In [54]:
z=joblib.load('/kaggle/working/model.pkl')
model.load_state_dict(z)

<All keys matched successfully>

In [57]:

model.eval()  
total_loss = 0
correct = 0
total = 0

with torch.no_grad(): 
    for x_batch, y_batch in dataloader:
        x_batch=x_batch.to(device)
        y_batch=y_batch.to(device)
        outputs = model(x_batch)
        loss = lossy(outputs, y_batch)
        total_loss += loss.item()
        probs = torch.sigmoid(outputs)
        preds = (probs > 0.5).float()
        correct += (preds == y_batch).sum().item()
        total += y_batch.numel()

accuracy = correct / total

print(f"Eval Loss: {total_loss/len(dataloader):.4f}")
print(f"Eval Accuracy: {accuracy * 100:.2f}%")

Eval Loss: 0.1142
Eval Accuracy: 95.95%


In [58]:
test=pd.read_csv('/kaggle/input/titanic/test.csv')
pid=test['PassengerId']
test=test.drop(columns=['PassengerId','Name','Ticket','Cabin'])
test["Age"] = test["Age"].fillna(test["Age"].mode()[0])
test["Fare"]=test["Fare"].fillna(test["Fare"].mode()[0])
test=test.dropna()
test

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,34.5,0,0,7.8292,Q
1,3,female,47.0,1,0,7.0000,S
2,2,male,62.0,0,0,9.6875,Q
3,3,male,27.0,0,0,8.6625,S
4,3,female,22.0,1,1,12.2875,S
...,...,...,...,...,...,...,...
413,3,male,21.0,0,0,8.0500,S
414,1,female,39.0,0,0,108.9000,C
415,3,male,38.5,0,0,7.2500,S
416,3,male,21.0,0,0,8.0500,S


In [59]:
test["Sex"]=test["Sex"].map(sex_id)
test["Embarked"]=test["Embarked"].map(emb_id)
test

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,0,34.5,0,0,7.8292,2
1,3,1,47.0,1,0,7.0000,0
2,2,0,62.0,0,0,9.6875,2
3,3,0,27.0,0,0,8.6625,0
4,3,1,22.0,1,1,12.2875,0
...,...,...,...,...,...,...,...
413,3,0,21.0,0,0,8.0500,0
414,1,1,39.0,0,0,108.9000,1
415,3,0,38.5,0,0,7.2500,0
416,3,0,21.0,0,0,8.0500,0


In [60]:
# ✅ Drop label if it exists
X_test = test.drop(columns=["Survived"], errors="ignore")

# ✅ Scale using the SAME scaler used in training
X_test_scaled = scaler.transform(X_test)

# ✅ Convert to torch tensor
x = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)

print("Input shape:", x.shape)

# ✅ Proper inference mode
model.eval()
with torch.no_grad():
    y = model(x)                # logits
    y = torch.sigmoid(y)        # probabilities
    y = (y > 0.5).int()         # binary predictions

print(y.squeeze())


Input shape: torch.Size([418, 7])
tensor([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1,
        1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
        1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0,
        0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0,
        1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1,
        1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1,
        0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
        0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
      

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [ ]:
print(pid)

In [62]:
final=pd.concat([pid,df],axis=1)

In [63]:
final

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,1
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [61]:
df=pd.DataFrame(y.to(torch.device('cpu')).numpy(),columns=["Survived"])

In [35]:
df

,Survived
0,0
1,0
2,0
3,0
4,0
...,...
413,0
414,1
415,0
416,0


In [64]:
final.to_csv("my_data.csv", index=False)
